In [ ]:
import pandas as pd
import numpy as np
import datetime

data = pd.read_csv('/Users/arshvetsoff/Desktop/DIPLOMA/atsenergo_2017_2023_1.csv')
data['is_mon'] = 0
data['is_sat'] = 0
data['is_sun'] = 0
data.index = data.timestamp
data.drop(columns=['timestamp'], inplace=True)
data.index = pd.to_datetime(data.index)
#start from 13-03-2020 till 12-03-2021 (365 days)
data = data.iloc[28008:28008+12264]
data = data.resample('D').mean()
data['is_mon'] = data.index.weekday == 0
data['is_sat'] = data.index.weekday == 5
data['is_sun'] = data.index.weekday == 6

In [ ]:
#Supress default INFO logging

import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

In [ ]:
data.head()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,5))
plt.plot(data['price'], label='target')
plt.xticks(rotation=60)
plt.axvline(x = data.iloc[397:397+2].index[0], color = 'g', linestyle='--')
plt.gcf().tight_layout()
plt.legend()
plt.show()

In [ ]:
train_df = data.iloc[:397]
train_df.index = train_df.index.astype('period[D]')
prices_train = np.log(train_df.price.values.astype(float))
mondays_train = train_df.is_mon.values.astype(int)
saturdays_train = train_df.is_sat.values.astype(int)
sundays_train = train_df.is_sun.values.astype(int)

test_df = data.iloc[397:]
test_df.index = test_df.index.astype('period[D]')
prices_test = np.log(test_df.price.values.astype(float))
mondays_test = test_df.is_mon.values.astype(int)
saturdays_test = test_df.is_sat.values.astype(int)
sundays_test = test_df.is_sun.values.astype(int)

len(prices_train), len(prices_test)

In [ ]:
train_data = {
    'N' : len(prices_train),
    'y' : list(prices_train),
    'D_Sat' : list(saturdays_train),
    'D_Sun' : list(sundays_train),
    'D_Mon' : list(mondays_train)
}

test_data = []
for i in range(len(prices_test)):
    test_data.append({
        'y' : prices_test[i],
        'D_Sat' : saturdays_test[i],
        'D_Sun' : sundays_test[i],
        'D_Mon' : mondays_test[i]
    })

In [ ]:
### FIT MODEL
from cmdstanpy import CmdStanModel
model = CmdStanModel(stan_file='/Users/arshvetsoff/Desktop/DIPLOMA/FUCKING_STAN_PREDICTIONS/prod_models/ats_1/dummy.stan')

In [ ]:
fit = model.sample(data=train_data, iter_sampling=120000, iter_warmup=800000, chains=1, adapt_delta=0.95)

In [ ]:
fit.save_csvfiles('/Users/arshvetsoff/Desktop/DIPLOMA/FUCKING_STAN_PREDICTIONS/prod_checkpoints/ats_1/dummy.csv')

In [ ]:
from cmdstanpy import from_csv

fit = from_csv('/Users/arshvetsoff/Desktop/DIPLOMA/FUCKING_STAN_PREDICTIONS/prod_checkpoints/ats_1/dummy.csv/dummy-20240510165050.csv')

In [ ]:
predicted_values = []

cur_fit = fit.summary().copy().loc[:, 'Mean'].to_dict()
print('copied initial fit')
train_data = {
    'N' : len(prices_train),
    'y' : list(prices_train),
    'D_Sat' : list(saturdays_train),
    'D_Sun' : list(sundays_train),
    'D_Mon' : list(mondays_train)
}

cnt = 0

for elem in test_data:
    train_data['N'] += 1
    train_data['y'].append(elem['y'])
    train_data['D_Sat'].append(elem['D_Sat'])
    train_data['D_Sun'].append(elem['D_Sun'])
    train_data['D_Mon'].append(elem['D_Mon'])
    
    prev_fit = cur_fit.copy()
    print('copied previous fit')
    cur_fit = model.sample(
        data=train_data,
        iter_sampling=7500,
        iter_warmup=5000,
        chains=1,
        inits=prev_fit  # Start from the last parameter means
    )
    predicted_values.append(cur_fit.stan_variable('prediction'))
    cur_fit = cur_fit.summary().loc[:, 'Mean'].to_dict()

In [ ]:
predicted_values